# INSTITUTO NACIONAL DE PESQUISAS ESPACIAIS
#Mestrado em Computação Aplicada (CAP)
# CAP-345-3 - Inteligência Artificial

* Aluno: Lourenço José Cavalcante Neto

These code was provide for [this](https://sites.google.com/view/ia-inpe-2022/) course. 

# **Aquisição dos dados para o Dataset que será usado no Treino e Teste do modelo de Rede Neural Convolucional** 

**Plota as imagens do ABI(Infravermelho) e GLM(Raios)**
---
**OBJETIVO:** Este Notebook serve plotar e salvar as imagens do canal infravermelho sobreposta com dados de relâmpagos do GLM de uma data específica, nas quais irão ser utilizadas no modelo de RNC.

**Dados de Entrada:** 

1.   **Dados do Sensor GLM:** Arquivos de descargas elétricas a cada 15 min. **Local:** Os dados são processados e fornecidos pelo CPTEC/INPE. Estão disponiveis e foram adquiridos [aqui](http://ftp.cptec.inpe.br/goes/goes16/goes16_web/glm_acumulado_nc/), **Extensão:** .nc, **Formato dos dados:** Netcdf.

2.   **Dados do Sensor ABI:** Canal 13 (Infravermelho - 10.8 µm) do sensor ABI a bordo do satélite GOES-16. **Local:** Os dados são processados e fornecidos pelo CPTEC/INPE. Estão disponiveis e foram adquiridos [aqui](http://ftp.cptec.inpe.br//goes/goes16/retangular/ch13/), **Extensão:**.nc, **Formato:** Netcdf.
---
**Dados de saída:**
Mapa espacial de Descargas elétricas
---
**Os seguintes procedimentos foram realizados nesse código:**
1.   Instalação das bibliotecas necessárias para geração dos mapas/imagens
2.   Importação das bibliotecas
3.   Montagem/Conexão com do google Drive
4.   Download dos dados através do servidor do CPTEC/INPE
5.   Plotagem e salvamento das imagens 



# **Instalação das bibliotecas necessárias**

In [ ]:
# Instalando o cartopy (usado para gerar Mapas)
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install --no-binary shapely shapely --force
!pip install cartopy

# Instalando o proplot (usado para gerar Mapas também)
!pip install proplot==0.6.4

!pip install geopandas

# **Download de arquivos/dados auxiliares**

In [ ]:
# Criando os diretórios de entrada e saida
import os
os.makedirs('input', exist_ok=True)
os.makedirs('arquivos_auxiliares', exist_ok=True)

# Baixando os arquivos de shapefile dos estados brasileiros
!wget -c https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2019/Brasil/BR/br_unidades_da_federacao.zip -P /content/arquivos_auxiliares/ 
print('\n')

# Descomprimindo o arquivo de shapefile dos estados brasileiros
!unzip -o  /content/arquivos_auxiliares/br_unidades_da_federacao.zip -d /content/arquivos_auxiliares/
print('\n')

# Baixa paleta de cores
!wget -c https://www.dropbox.com/s/t7b8x2i3gnsq8gv/cpt_convert.py 
!wget -c https://www.dropbox.com/s/74vlk75bkh3fzaz/IR4AVHRR6.cpt

# **Importa as bibliotecas**

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import proplot as plot
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from cartopy.feature import NaturalEarthFeature
import matplotlib.pyplot as plt
import os
import matplotlib
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from datetime import datetime, timedelta

import matplotlib
matplotlib.rcParams.update({'font.size':22})
from matplotlib import pyplot as plt
from cpt_convert import loadCPT # Importando a função CPT convert 
from matplotlib.colors import LinearSegmentedColormap # interpolação linear para as cores dos mapas
from netCDF4 import Dataset  
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")


# **Conexão com o Drive e definição do caminho da pasta para as imagens**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define o caminho do drive. Pasta para a saída das Figuras geradas que serão usadas na CNN
#path_fig_abi_glm = f'/content/drive/MyDrive/MestradoCAP/DADOS_CAP354/database_for_cnn/training_set/presence_of_electrical_discharges/'
#path_fig_abi = f'/content/drive/MyDrive/MestradoCAP/DADOS_CAP354/database_for_cnn/training_set/no_electrical_discharges/'
path_outputGerais = f'/content/drive/MyDrive/MestradoCAP/DADOS/outputGerais/'

# **Importa funções**

In [ ]:
import geopandas as gpd
def evm_plot_states2():
    estados_BR = gpd.read_file('/content/arquivos_auxiliares/BR_UF_2019.shp', geom_col='geometry', encoding='utf-8')
    estado = 'Amazonas'
    estado_cd_uf = 13
    estado_poly = estados_BR[estados_BR.NM_UF == estado]
    shapefile = list(estado_poly.geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black',facecolor='none', linewidth=0.6) 


In [ ]:
#----------------------------------------------------------------------------------- 
# Função que plota os Estados 
#----------------------------------------------------------------------------------- 
def evm_plot_states():
    shapefile = list(shpreader.Reader('/content/drive/MyDrive/MestradoCAP/DADOS/auxiliares/shapefile/BR_UF_2021/BR_UF_2021.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black',facecolor='none', linewidth=0.8)

#----------------------------------------------------------------------------------- 
# Função que plota Amazonas no mapa 
#----------------------------------------------------------------------------------- 
def evm_plot_amazon():
    shapefile_amazon = list(shpreader.Reader('/content/drive/MyDrive/MestradoCAP/DADOS_CAP354/auxiliares/shapefile/Sede_Mun_Amazonia_Legal_2020.shp').geometries())
    ax.add_geometries(shapefile_amazon, ccrs.PlateCarree(), edgecolor='red',facecolor='none', linewidth=0.5)
  

# **PROCESSAMENTO DOS DADOS:**
Nesta etapa serão utilizados os dados de **relâmpagos** do sensor GLM e **temperatura de brilho** do canal infravermelho do sensor ABI, ambos pertencentes ao satélite GOES-16. Serão produzidas as seguintes figuras:

1.   **Mapa espacial** da temperatua do canal infravermelho do satélite e relâmpagos do satélite GOES-16.

Vamos definir a data e a hora dos dados para download e posterior geração da imagem.

In [ ]:
ano = '2019'
mes = '01'
dia = '21'
hor = '16'
minu = '00'

## **Baixando os dados do sensor ABI:**

In [ ]:
# Endereço do FTP do CPTEC-INPE
ftp_cptec = 'ftp.cptec.inpe.br'
# ---------------------------------------------------------- #
#              BAIXANDO DADOS DO ABI
# ---------------------------------------------------------- #
# Nome do arquivo
file_ir =  f'{ftp_cptec}/goes/goes16/retangular/ch13/{ano}/{mes}/S10635346_{ano}{mes}{dia}{hor}{minu}.nc'

# Download dos arquivos
!wget -c {file_ir} -P /content/input/

## **Baixando os dados do sensor GLM:**

In [ ]:
# ---------------------------------------------------------- #
#              BAIXANDO DADOS DO GLM 
# ---------------------------------------------------------- #
# S11635949_202009010000.nc	
path_glm = '/goes/goes16/goes16_web/glm_acumulado_nc/' ; path_total = f'{ftp_cptec}{path_glm}'

# primeiro arquivo: 5min
#basename_glm_5min = f'S11635949_{ano}{mes}{dia}{hor}{minu}.nc'

# segundo arquivo: 10min
date_10min = str(datetime(int(ano), int(mes), int(dia), int(hor), int(minu)) + timedelta(minutes=5)) # calcula a imagem + 5min
ano_10min = datetime.strptime(date_10min, '%Y-%m-%d %H:%M:%S').strftime('%Y')  # extrai o ano 
mes_10min = datetime.strptime(date_10min, '%Y-%m-%d %H:%M:%S').strftime('%m')  # extrai o mes 
dia_10min = datetime.strptime(date_10min, '%Y-%m-%d %H:%M:%S').strftime('%d')  # extrai o dia 
hor_10min = datetime.strptime(date_10min, '%Y-%m-%d %H:%M:%S').strftime('%H')  # extrai o hora 
minu_10min = datetime.strptime(date_10min, '%Y-%m-%d %H:%M:%S').strftime('%M') # extrai o minuto
basename_glm_10min = f'S11635949_{ano_10min}{mes_10min}{dia_10min}{hor_10min}{minu_10min}.nc'

 # path + nome dos arquivos
#file_glm_5min = f'{path_total}{ano}/{mes}/{basename_glm_5min}'
file_glm_10min = f'{path_total}{ano_10min}/{mes}/{basename_glm_10min}'

# Download dos arquivos
#!wget -c {file_glm_5min} -P /content/input/
!wget -c {file_glm_10min} -P /content/input/

# **Loop - Plotando o mapa espacial: ABI CH13 + GLM Flashes de Janeiro 2019** 


In [ ]:
list_min_abi_glm = ['00','10','20','30','40','50']
abi_data_list = []
glm_data_list = []

#ABI
os.chdir('/content/drive/MyDrive/MestradoCAP/DADOS/abi/nc/2019/1/')
full_direc_abi = os.listdir()

  
#GLM
os.chdir('/content/drive/MyDrive/MestradoCAP/DADOS/glm/nc/2019/1/')
full_direc_glm = os.listdir()


In [ ]:
conta_plot = 0
for abi_nc in full_direc_abi:
  for glm_nc in full_direc_glm:
    abi_nc_res = abi_nc[:-3][-12::]
    glm_nc_res = glm_nc[:-3][-12::]
    
    if (abi_nc_res[:8] == '20190114' or abi_nc_res[:8] == '20190115') and (abi_nc_res == glm_nc_res):
    #if (abi_nc_res == glm_nc_res):
      year = glm_nc_res[0:4]
      month = glm_nc_res[4:6]
      day = glm_nc_res[6:8]
      hour = glm_nc_res[8:10]
      minute = glm_nc_res[10:12]

      if (glm_nc_res[-2::] == '00' or glm_nc_res[-2::] == '15' or glm_nc_res[-2::] == '30' or glm_nc_res[-2::] == '45'):
      
        #glm_nc_15min = str(int(p1_nc),int(p2_nc) + timedelta(minutes=15))

        if(glm_nc_res[-2::] == '45'):
          
          glm_nc_15min_p1 = str(int(glm_nc_res[-3::]) + 55)
          glm_nc_15min = str(glm_nc_res[:9])+''+glm_nc_15min_p1
          
        else:
          glm_nc_15min = str(int(glm_nc_res[:12]) + 15)
          #print(glm_nc_15min)

        conta_plot = conta_plot + 1

        #basename = os.path.basename(os.path.splitext(file_ir)[0]) 
        file = f'/content/drive/MyDrive/MestradoCAP/DADOS/abi/nc/2019/1/{abi_nc}'

        # Leitura do arquivo ABI
        imagem = xr.open_dataset(file)

        # Extração dos limites: latitudes e longitudes
        latmin, latmax, lonmin, lonmax = float(imagem['lat'][0]), float(imagem['lat'][-1]), float(imagem['lon'][0]), float(imagem['lon'][-1])
        # lats/lons do quadrado limitando o Brasil
        #latmin1, latmax1, lonmin1, lonmax1 =  -35, 7, -75, -32
        latmin1, latmax1, lonmin1, lonmax1 =  2.4, -10, -74, -55.11 #lats/lons do quadrado limitando ao estado do amazonas
        #latmin1, latmax1, lonmin1, lonmax1 =  -6, -10, -74, -70 #lats/lons do quadrado limitando as imagens das amostras com e sem raios

        # Transformação da temperatura lida para Celsius
        imagem = (imagem['Band1']/100.)-273.15

        # Inverte a matriz
        imagem = np.flipud(imagem)

        # Configurações da plotagem da Figura
        fig, ax = plot.subplots(axwidth=7, axheight=7, tight=True, proj='pcarree')
        
        # Add coastlines, borders and gridlines
        #ax.coastlines(resolution='10m', color='white', linewidth=0.2, zorder=1)
        # add the geographic boundaries
        l = NaturalEarthFeature(category='cultural', name='admin_0_countries', scale='110m', facecolor='none')
        ax.add_feature(l, edgecolor='', linewidth=0.25)

        #----------------------------------------------------------------------------------- 
        # Caso queiramos plotar os Estados Brasieliros no mapa
        #----------------------------------------------------------------------------------- 
        '''
        shapefile = list(shpreader.Reader('/content/drive/MyDrive/MestradoCAP/DADOS/auxiliares/shapefile/BR_UF_2021/BR_UF_2021.shp').geometries())
        ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black',facecolor='none', linewidth=0.8)
        '''
        #----------------------------------------------------------------------------------- 
        # Caso queiramos plotar o Estado do Amazonas no mapa 
        #----------------------------------------------------------------------------------- 
        
        shapefile_amazon = list(shpreader.Reader('/content/drive/MyDrive/MestradoCAP/DADOS/auxiliares/shapefile/AM/Sede_Mun_Amazonia_Legal_2020.shp').geometries())
        ax.add_geometries(shapefile_amazon, ccrs.PlateCarree(), edgecolor='red',facecolor='none', linewidth=1.0) 
        

        # Aqui definimos o formato da imagems
        ax.format(coast=False, borders=False, innerborders=False,labels=False, latlim=(latmin1,latmax1), lonlim=(lonmin1,lonmax1), fontsize=9, 
                  small='25px', large='25px')
        # Choose the plot size (width x height, in inches)

        #Carrega tabela de cores do Infravermelho
        cpt_ir = loadCPT('/content/drive/MyDrive/MestradoCAP/DADOS/auxiliares/paleta_cores/IR4AVHRR6.cpt')
        cpt_convert_ir = LinearSegmentedColormap('cpt_ir', cpt_ir)
        # limites da paleta de cores
        vmin_ir = -103.0 
        vmax_ir = 105

        mean_temperature_ABI = round(np.mean(imagem),2) #Temperatura média 

        # Aqui é plotada e variável Infravermelho (ABI)
        map1 = ax.imshow(imagem, vmin=vmin_ir, vmax=vmax_ir, cmap=cpt_convert_ir, extent=[lonmin, lonmax, latmin, latmax],
                        levels=plot.arange(vmin_ir, vmax_ir, 1.0))

        #---------------------------------------------#
        #              plota flashes GLM (15min)
        #---------------------------------------------#

         # Leitura do arquivo GLM

        #Caso queiramos verificar os flashes GLM que ocorreram 15min após o minuto do ABI atual,
        #quando formos trabalhar com o modelo para previsão de descragas elétricas
        #glm_15min = xr.open_dataset(f'/content/drive/MyDrive/MestradoCAP/DADOS/glm/nc/2019/1/S11635949_{glm_nc_15min}.nc') 
        
        #Caso queiramos verificar os flashes GLM que ocorreram no mesmo minuto do ABI atual.
        '''
        glm_15min = xr.open_dataset(f'/content/drive/MyDrive/MestradoCAP/DADOS/glm/nc/2019/1/{glm_nc}')
        df = glm_15min.to_dataframe().reset_index().dropna()
        ax.scatter(df['lon'].values, df['lat'].values, transform=ccrs.PlateCarree(), marker="X",alpha=1,s=5, color='fuchsia')  
        
        
        qtd_flash_GLM_atual = df['flash'].sum()
        qtd_flash_GLM_atual = '{:f}'.format(qtd_flash_GLM_atual)
        '''
        #qtd_flash_GLM_atual = int(qtd_flash_GLM_atual)

        
        # salva figura
        plt.axis("off")
        #fig.save(f'{path_fig_abi_glm}{conta_plot}_g16_abi_glm_{year}{month}{day}_{hour}{minute}.png', dpi=300, bbox_inches='tight')
        fig.save(f'{path_outputGerais}{conta_plot}_g16_abi_{year}{month}{day}_{hour}{minute}.png', dpi=300, bbox_inches='tight')
        
        print(conta_plot,' :Imagem gerada e salva com sucesso. Qtd Flash:',qtd_flash_GLM_atual,' Temp. Média:',mean_temperature_ABI)
        print(conta_plot,' - ',abi_nc_res,' :Imagem gerada e salva com sucesso. Temp. Média:',mean_temperature_ABI)

        # exibe a figura na tela
        #plt.axis("off")
        plt.grid(False)
        plot.show()
  #Aqui podemos limitar o número de imagens que queremos gerar/salavar
  if(conta_plot == 1):
    break